# Setup

In [1]:
from google.colab import drive
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import numpy as np

In [2]:
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/relevance/processed_data_with_stimuli.csv')
df
# df = pd.read_json(f'/content/drive/MyDrive/relevance/processed.jsonl', orient="records", lines=True)
# df_stim = pd.read_csv('/content/drive/MyDrive/relevance/relevance_stimuli.csv')

In [ ]:
df.columns

In [3]:
cols = [

    # Trial Metadata
    # 'submission_id',
    # 'StimID',
    'RowID',
    'AnswerCertainty',
    # 'AnswerPolarity',
    'ContextType',
    'Answer',
    'Context',
    'YourQuestionIntro',
    'YourQuestion',

    # Raw Judgments
    'pri',
    'pos',
    'conf_pri',
    'conf_pos',
    'rel',

    # First Order Measures
    # 'bch',
    # 'ech',
    # 'klu',
    # 'bfu',

    # Second Order Measures
    # '2ord_bch',
    # 'beta_ech',
    # 'beta_klu',
    # 'beta_bfu',
    # 'beta_bch',

    # Ranks / Rank diffs
    'rel_rank',

    # 'bch_rank_diff',
    # 'ech_rank_diff',
    # 'klu_rank_diff',
    'bfu_rank_diff',

    # '2ord_bch_rank_diff',
    # 'beta_ech_rank_diff',
    # 'beta_klu_rank_diff',
    'beta_bfu_rank_diff',
    # 'beta_bch_rank_diff',

]

# General analysis

Let's find examples where there is high relevance but low belief change

In [ ]:
df[df.apply(lambda x: x.bch < 0.5 and x["2ord_bch"] < 4 and x.rel > 0.8 and x.AnswerCertainty == "exhaustive", axis=1)][cols]

Let's find even more extreme examples

In [ ]:
df[df.apply(lambda x: x.bch < 0.2 and x["2ord_bch"] < 3 and x.rel > 0.8, axis=1)][cols]

Let's find the school bus example

In [ ]:
df[df.apply(lambda x: x.YourQuestion == "Is school canceled?", axis=1)][cols]

Here are examples with low belief change but high relevance for non-answers:

In [ ]:
edf[df.apply(lambda x: x.bch < 0.1 and x["2ord_bch"] < 2 and x.rel > 0.3 and x.AnswerCertainty == "non_answer", axis=1)][cols]

# Does Prior Matter?

In [6]:
cols = [

    # Trial Metadata
    # 'submission_id',
    'StimID',
    # 'RowID',
    'AnswerCertainty',
    'AnswerPolarity',
    'ContextType',
    'Answer',
    'Context',
    'YourQuestionIntro',
    'YourQuestion',

    # Raw Judgments
    'pri',
    'pos',
    'conf_pri',
    'conf_pos',
    'rel',

    # First Order Measures
    'bch',
    # 'ech',
    # 'klu',
    # 'bfu',

    # Second Order Measures
    # '2ord_bch',
    # 'beta_ech',
    # 'beta_klu',
    # 'beta_bfu',
    # 'beta_bch',

    # Ranks / Rank diffs
    # 'rel_rank',

    # 'bch_rank_diff',
    # 'ech_rank_diff',
    # 'klu_rank_diff',
    # 'bfu_rank_diff',

    # '2ord_b ch_rank_diff',
    # 'beta_ech_rank_diff',
    # 'beta_klu_rank_diff',
    # 'beta_bfu_rank_diff',
    # 'beta_bch_rank_diff',

]

In [7]:
df_ = df[cols]
df_ = df_[df_.apply(lambda x:
                    x.AnswerCertainty != "non_answer"
                    # and x.ContextType != "neutral"
                    , axis=1)]
df_["expected_belief_change"] = df_.apply(lambda x:
                                          "low" if x.AnswerPolarity == x.ContextType
                                          else "mid" if x.ContextType == "neutral"
                                          else "high"
                                          , axis=1)
df_["AnswerCondition"] = df_.apply(lambda x: x.AnswerCertainty + "-" + x.AnswerPolarity, axis=1)
# df_.set_index(["StimID", "AnswerCertainty", "expected_belief_change"])
df_

This is how the low- mid- and high-expected belief change relevance scores vary for ALL answers in the dataset.

In [ ]:
sns.catplot(data=df_, col="AnswerCondition", row="StimID",
            y="rel", hue="expected_belief_change", kind="violin", hue_order=["low", "mid", "high"])

Now we take the mean for each `AnswerCondition/StimID pair` and compute the difference for `high` and `low` values of `expected_belief_change`.

In [14]:
df_diff = df_[["StimID", "expected_belief_change", "AnswerCondition", "AnswerCertainty", "rel", "bch"]]
df_diff = df_diff.groupby(["StimID", "AnswerCondition", "AnswerCertainty", "expected_belief_change"]).mean().unstack()

df_diff["rel_difference"] = df_diff.apply(lambda x:
                                      x[("rel", "high")] - x[("rel", "low")]
                                      , axis=1)
df_diff["bch_difference"] = df_diff.apply(lambda x:
                                      x[("bch", "high")] - x[("bch", "low")]
                                      , axis=1)
df_diff = df_diff.reset_index().droplevel(1, axis=1).drop(["rel", "bch"], axis=1)
df_diff

Plot by faceting on answer certainty

In [32]:
g = sns.displot(data=df_diff,
                hue="AnswerCertainty",
                x="rel_difference",
                hue_order=["low_certainty", "high_certainty", "exhaustive"],
                kind="kde",
                palette="viridis",
                fill=True,
                aspect=2,
                height=3)
g.axes.flatten()[0].set_xlabel(
    """Average relevance of high expected belief change condition
        minus average relevance of low expected belief change condition,
        by answer.""")
g.set(xlim=(-0.4, 0.4))
plt.axvline(x = 0, color = 'orange', linestyle="dashed")

plt.savefig("/content/drive/MyDrive/relevance/figures/impact_of_prior_by_answer_certainty.pdf", bbox_inches="tight")

Plot with no faceting

In [33]:
g = sns.displot(data=df_diff,
                # hue="AnswerCertainty",
                x="rel_difference",
                # hue_order=["low_certainty", "high_certainty", "exhaustive"],
                kind="kde",
                palette="viridis",
                fill=True,
                aspect=2,
                height=3)
g.axes.flatten()[0].set_xlabel(
    """Average relevance of high expected belief change condition
        minus average relevance of low expected belief change condition,
        by answer.""")
g.set(xlim=(-0.4, 0.4))
plt.axvline(x = 0, color = 'orange', linestyle="dashed")

plt.savefig("/content/drive/MyDrive/relevance/figures/impact_of_prior.pdf", bbox_inches="tight")

Plotting belief change difference in the same way

In [31]:
g = sns.displot(data=df_diff,
                hue="AnswerCertainty",
                x="bch_difference",
                hue_order=["low_certainty", "high_certainty", "exhaustive"],
                kind="kde",
                palette="viridis",
                fill=True,
                aspect=2,
                height=3)
g.axes.flatten()[0].set_xlabel(
    """Average relevance of high expected belief change condition
        minus average relevance of low expected belief change condition,
        by answer.""")
g.set(xlim=(-0.4, 1))
plt.axvline(x = 0, color = 'orange', linestyle="dashed")

plt.savefig("/content/drive/MyDrive/relevance/figures/impact_of_prior_by_answer_certainty.pdf", bbox_inches="tight")

2d KDE

In [34]:
g = sns.displot(data=df_diff,
                # col="AnswerCertainty",
                x="rel_difference",
                y="bch_difference",
                # hue_order=["low_certainty", "high_certainty", "exhaustive"],
                kind="kde",
                palette="viridis",
                fill=True,
                aspect=2,
                height=3)
g.axes.flatten()[0].set_xlabel(
    """Average relevance of high expected belief change condition
        minus average relevance of low expected belief change condition,
        by answer.""")
g.set(xlim=(-0.4, 0.4))
plt.axvline(x = 0, color = 'orange', linestyle="dashed")
plt.axhline(y = 0, color = 'orange', linestyle="dashed")
plt.savefig("/content/drive/MyDrive/relevance/figures/impact_of_prior.pdf", bbox_inches="tight")

# Metrics

## BFU

In [ ]:
df_bfu = df.sort_values(by="bfu_rank_diff")[cols + ["bfu", "bfu_rank_diff"]]
# df_bfu
df_bfu = df_bfu[df_bfu.apply(lambda x: x.bfu > 0 and x.rel < 1, axis=1)]

In [ ]:
df_filter = df[df.apply(lambda x: not (x["bch"] == 0 and x["2ord_bch"] == 0 and x["AnswerCertainty"] != "non_answer"), axis=1)]
sns.relplot(data=df_filter, x="pri", y="pos", hue="bfu_rank_diff", palette="coolwarm", alpha=0.5)

In [ ]:
ranks = [

    # 'rel_rank',

    'bch_rank_diff',
    'ech_rank_diff',
    'klu_rank_diff',
    'bfu_rank_diff',

    # '2ord_bch_rank_diff',
    'beta_ech_rank_diff',
    'beta_klu_rank_diff',
    'beta_bfu_rank_diff',
    'beta_bch_rank_diff',
]

other_cols = [

    'pri',
    'pos',
    'rel',
    'conf_pos',
    'conf_pri',
]


df_plot = df_filter[ranks + other_cols].set_index(other_cols).stack().reset_index().rename({"level_5": "rank_type", 0: "rank"}, axis=1)

def extract_metric(name):
    if "ech" in name:
        return "Entropy Change"
    elif "klu" in name:
        return "KL Utility"
    elif "bch" in name:
        return "Belief Change"
    elif "bfu" in name:
        return "Bayes Factor Utility"

def extract_order(name):
    if "beta" in name:
        return "Second Order"
    else:
        return "First Order"

df_plot["metric"] = df_plot["rank_type"].apply(extract_metric)
df_plot["order"] = df_plot["rank_type"].apply(extract_order)
df_plot

Plotting just the relevance score

In [ ]:
g = sns.relplot(data=df,
                x="pri",
                y="pos",
                hue="rel",
                palette="viridis",
                alpha=0.2,
                s=250,
                height=4,
                edgecolor='none'
            )

# # Rotate the row titles (y-axis labels)
# for ax, row in zip(g.axes[:,0], df_plot["order"].unique()):
#     ax.annotate(row, xy=(0, 0.5), xytext=(-ax.yaxis.labelpad - 5, 0),
#                 xycoords=ax.yaxis.label, textcoords='offset points',
#                 size='large', ha='right', va='center', rotation=90)
#     ax.set_ylabel("Posterior Probability")

# # Add column titles at the top and remove others
# for ax, col in zip(g.axes[0], df_plot["metric"].unique()):
#     ax.set_title(col)
# for ax in g.axes[1]:
#     ax.set_title("")
#     ax.set_xlabel("Prior Probability")

# g._legend.set_title('Δ Rank')

In [ ]:
def plot_rank_diffs(data_subset):
    g = sns.relplot(data=data_subset,
                    x="pri",
                    y="pos",
                    hue="rank",
                    col="metric",
                    row="order",
                    palette="Spectral",
                    alpha=0.2,
                    s=100,
                    height=2,
                    edgecolor='none'
                )

    # Rotate the row titles (y-axis labels)
    for ax, row in zip(g.axes[:,0], df_plot["order"].unique()):
        ax.annotate(row, xy=(0, 0.5), xytext=(-ax.yaxis.labelpad - 5, 0),
                    xycoords=ax.yaxis.label, textcoords='offset points',
                    size='large', ha='right', va='center', rotation=90)
        ax.set_ylabel("Posterior Probability")

    # Add column titles at the top and remove others
    for ax, col in zip(g.axes[0], df_plot["metric"].unique()):
        ax.set_title(col)
    for ax in g.axes[1]:
        ax.set_title("")
        ax.set_xlabel("Prior Probability")

    g._legend.set_title('Δ Rank')

plt.savefig("/content/drive/MyDrive/relevance/figures/rank_difference.pdf")

Let's redo this plot but only focus on the corners

In [ ]:
p = 0.2
q = 1-p
r = 0.5 / p
df_extreme = df_plot[df_plot.apply(lambda x:
                                    (x.pri < p and x.pos < p) or
                                    (x.pri < p and x.pos > q) or
                                    (x.pri > q and x.pos < p) or
                                    (x.pri > q and x.pos > q),
                                axis=1)]

df_extreme["pri"] = df_extreme.pri.apply(lambda x: r * x if x < p else 1 - (r * (1-x)))
df_extreme["pos"] = df_extreme.pos.apply(lambda x: r * x if x < p else 1 - (r * (1-x)))
plot_rank_diffs(df_extreme)

Let's pull out examples where there was small first-order belief change, but high second-order belief change

In [ ]:
df_small_bc = df_plot[df_plot.apply(lambda x:
                                    abs(x.pos - x.pri) < 0.2 and x.rel > 0.5,
                                    axis=1
                                    )]
g = sns.relplot(data=df_small_bc,
                x="pri",
                y="pos",
                hue="rank",
                col="metric",
                row="order",
                palette="Spectral",
                alpha=0.2,
                s=100,
                height=2,
                edgecolor='none'
            )

# Rotate the row titles (y-axis labels)
for ax, row in zip(g.axes[:,0], df_plot["order"].unique()):
    ax.annotate(row, xy=(0, 0.5), xytext=(-ax.yaxis.labelpad - 5, 0),
                xycoords=ax.yaxis.label, textcoords='offset points',
                size='large', ha='right', va='center', rotation=90)
    ax.set_ylabel("Posterior Probability")

# Add column titles at the top and remove others
for ax, col in zip(g.axes[0], df_plot["metric"].unique()):
    ax.set_title(col)
for ax in g.axes[1]:
    ax.set_title("")
    ax.set_xlabel("Prior Probability")

g._legend.set_title('Δ Rank')

In [ ]:
# df_small_bc = df_plot[df_plot.apply(lambda x:
#                                     abs(x.pos - x.pri) < 0.1 and x.rel > 0.5 and x.,
#                                     axis=1
#                                     )]
# df_small_bc
print(df.columns)
df[df.apply(lambda x:
            abs(x.pos - x.pri) < 0.1 and
            x.rel > 0.5 and
            x.klu_rank_diff < -500,
            axis=1
            )][cols + ["klu_rank_diff", "beta_klu_rank_diff", "bfu_rank_diff", "beta_bfu_rank_diff"]]

Let's try to plot for one metric, separating data points by change in confidence

In [ ]:
def conf_coarse(conf):
    return {
        1: "1-2",
        2: "1-2",
        3: "3-4",
        4: "3-4",
        5: "5",
        6: "6",
        7: "7"
    }[conf]

df_plot["conf_pri_coarse"] = df_plot["conf_pri"].apply(conf_coarse)
df_plot["conf_pos_coarse"] = df_plot["conf_pos"].apply(conf_coarse)

In [ ]:
df_beta_klu = df_plot[df_plot["rank_type"]=="beta_klu_rank_diff"]
g = sns.relplot(data=df_beta_klu,
                x="pri",
                y="pos",
                hue="rank",
                col="conf_pri_coarse",
                row="conf_pos_coarse",
                row_order=["7", "6", "5", "3-4", "1-2"],
                palette="Spectral",
                alpha=0.5,
                s=50,
                edgecolor='none',
                height=1,
                aspect=1
                )
g.set_titles("")
g.set(ylim=(0, 1), xlim=(0, 1))
plt.subplots_adjust(left=0.1, right=0.8, top=0.9, bottom=0.1, wspace=0.2, hspace=0.2)
